# Day 17

link: https://adventofcode.com/2022/day/17

ข้อนี้เหมือนจำลองเกมแนว Tetris (แต่บาง piece มันมี 5 tiles จะเรียก tetris ก็เรียกได้ไม่เต็มปากเพราะตามชื่อแล้ว Tetris มันต้องมีชิ้นละ 4 tiles)

เราจะ encode game state ด้วย integer list โดนแทนแต่ละ row ด้วยเลข 1 ตัว เราแทน empty cell ด้วย 0, occupied cell ด้วย 1

เราใส่ 127 (1111111₂) ไว้เป็น row ล่างสุด (ก็คือ item แรกใน game state list) ทำหน้าที่เป็นพื้น เพื่อให้ง่ายตอนทำ collision check

ส่วนหินทั้ง 5 รูปแบบ จะ encode ได้แบบนี้

In [1]:
rocks = [
  [30],
  [8, 28, 8],
  [28, 4, 4],
  [16, 16, 16, 16],
  [24, 24]
]
nil

เราสร้าง method `simulate` เพื่อดูว่าหินจะตกไปที่ตำแหน่งไหน
- `a` คือ game state
- `rocks` คือหิน 5 รูปแบบตามข้างบน
- `ri` คือ index ของหินที่จะโผล่มาในรอบนี้
- `winds` คือทิศทางของลมทั้งหมด ซึ่งเรารับมาจาก input โดยตรง
- `wi` คือ index ของลมเมื่อเริ่มรอบนี้

แรกสุดเราเพิ่ม rows ว่างเข้าไปใน game state จำนวน 3 rows แล้วตั้ง row index ของหิน ไว้ที่ `fi`
จากนั้นก็ simulate ตามการเคลื่อนไหวของลม โดยที่เราต้องทำ collision check ว่าหินสามารถขยับตามลมได้หรือไม่
ถ้าได้ก็ขยับตำแหน่งแกน x ของหิน ซึ่งใน encoding ของเรา การขยับซ้ายก็คือคูณสอง ขยับขวาก็คือหารสอง
ส่วน collision check ก็คือการทำ bitwise and ซึ่งถ้าไม่มีอะไรชนกัน มันจะออกมาเป็น 0

จากนั้นเราทำ collision check อีกทีเพื่อดูว่าหินสามารถตกลงมาได้มั้ย ถ้าได้ก็ขยับ `fi` แล้ววนซ้ำใหม่ 
ถ้าไม่ได้ก็แสดงว่าเราได้ตำแหน่งสุดท้ายของหินแล้ว เราก็ merge หินลงไปใน game state โดยการทำ bitwise or แล้วก็ clear empty rows ออกจาก game state

method นี้ return game state ที่อัพเดตแล้ว และ index ของลมในรอบถัดไป

In [2]:
input = IO.read('in17.txt')

def simulate(a, rocks, ri, winds, wi)
  rock = rocks[ri]
  fi = a.size + 3
  a += [0] * (rock.size + 3)

  loop {
    if winds[wi] == '>'
      rock = rock.map{|x| x >> 1} if rock.each_with_index.all?{|row, i| row[0] == 0 && (row >> 1) & a[fi + i] == 0}
    else
      rock = rock.map{|x| x << 1} if rock.each_with_index.all?{|row, i| row[6] == 0 && (row << 1) & a[fi + i] == 0}
    end
    wi = (wi+1) % winds.size

    break if rock.each_with_index.any?{|row, i|
      row & a[fi - 1 + i] > 0
    }
    fi -= 1
  }

  rock.size.times{|j| a[fi+j] |= rock[j] }
  a.pop while a[-1] == 0
  [a, wi]
end
nil

## Part 1

ใน part แรก เราแค่ simulate หิน 2022 ก้อน คำตอบก็คือจำนวน rows ใน game state (ไม่นับพื้นใน row ล่างสุด)

In [3]:
a = [127]
ri = 0
wi = 0

2022.times{
  a, wi = simulate(a, rocks, ri, input, wi)  
  ri = (ri + 1) % 5
}
puts a.size - 1

3166


## Part 2

part นี้ให้เราหาความสูงหลังจากหินก้อนที่ 10^12 ซึ่งเราไม่มีทาง simulate ทีละก้อนได้แน่ๆ

จุดสังเกตก็คือ เนื่องจากทิศทางลมมีความยาวจำกัด (ประมาณ 10000) และหินก็มีจำนวนจำกัด (5 รูปแบบ)
ดังนั้นมันจะต้องวนกลับมาที่ state เดิมภายในประมาณ 50000 steps
ถ้าเราหาขนาดของ loop นี้ได้ เราก็จะเหลืองานที่ต้อง simulate แค่ก่อนหน้าเข้า loop, ใน loop, และหลังจาก loop

ที่พูดมาข้างบนนี้ก็ไม่ครบถ้วนซะทีเดียว เพราะอีกตัวแปรหนึ่งที่มีผลต่อ state นอกจากรูปแบบของหินและลมแล้ว ก็ยังมี game state เองอีกด้วย
เราพอจะ assume ได้ว่า game state ที่มีผลมันมีแค่ row บนๆ ไม่กี่ rows เท่านั้น แต่จะเก็บเป็นส่วนหนึ่งของ state ด้วยก็อาจจะซับซ้อนเกินจำเป็น
วิธีที่ง่ายกว่าคือ สังเกตว่ามันจะมีแค่ช่วงหินก้อนแรกๆ ที่อยู่ใน state ที่ไม่ซ้ำกับ state ที่เป็นส่วนหนึ่งของ loop
หมายความว่า พอรันไปซักพักเราจะเข้าสู่ส่วนที่เป็น loop เพราะงั้นเราก็แค่รัน warmup ไปสักพัก แค่พอให้มันพ้นช่วงแรกไป
จากนั้นก็ใช้แค่หินกับลมเป็น state ตามที่คิดไว้แต่แรก

In [4]:
a = [127]
rock_count = 0
wi = 0

t = 10**12
warmup = 1000
t -= warmup

warmup.times{
  a, wi = simulate(a, rocks, rock_count % 5, input, wi)  
  rock_count += 1
}

records = input.size.times.map{ Array.new(5) }

ri = rock_count % 5
until records[wi][ri]
  records[wi][ri] = [rock_count, a.size - 1]
  a, wi = simulate(a, rocks, ri, input, wi)  
  rock_count += 1
  ri = rock_count % 5 
end

loop_size = rock_count - records[wi][ri][0]
rows_per_loop = a.size - 1 - records[wi][ri][1]
rows_pre_loop = records[wi][ri][1]

loops, remain = t.divmod(loop_size)

remain.times{
  a, wi = simulate(a, rocks, rock_count % 5, input, wi)  
  rock_count += 1 
}

rows_post_loop = a.size - 1 - (rows_pre_loop + rows_per_loop)

puts rows_pre_loop + rows_per_loop * loops + rows_post_loop

1577207977186
